# Setup

In [1]:
!pip install --upgrade pip
!pip install -U langchain langchain-community tavily-python faiss-cpu huggingface_hub langgraph langchain_experimental

## Setup Tavily

In [5]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = "MY API KEY"

search = TavilySearchResults(max_results=2)

## Non-working example

The changes I had to make to the code was:

ollama_functions.py
```python

# Update to the default system template adding in two additional lines. I found that llama3 was not always respecting the schema of the tools.
DEFAULT_SYSTEM_TEMPLATE = """You have access to the following tools:

{tools}

You must always select one of the above tools and respond with only a JSON object matching the following schema:

{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}

Double check that in your json, you only have `tool` and `tool_input`.
Also Double check that the tool_input is an object and matches the parameters for the selected tool, matching the tool's JSON schema.
"""
```

# Update to the _generate message
I had to remove the if check in the _generate function as the check for if it was a is_pydantic_class always failed
```python
    # if _is_pydantic_class(functions[0]):
    functions = [convert_to_ollama_tool(fn) for fn in functions]
    functions.insert(0, DEFAULT_RESPONSE_FUNCTION)
```

# update the ollama tool conversion function
def convert_to_ollama_tool(tool: Any) -> Dict:
    """Convert a tool to an Ollama tool."""

    if _is_pydantic_class(tool.__class__):
        schema = tool.__dict__["args_schema"].schema()
        definition = {"name": tool.name, "properties": schema["properties"]}
        if "required" in schema:
            definition["required"] = schema["required"]

        return definition
    raise ValueError(
        f"Cannot convert {tool} to an Ollama tool. {tool} needs to be a Pydantic model."
    )
```

chat_models/ollama.py
```python
def _convert_messages_to_ollama_messages(
        self, messages: List[BaseMessage]
    ) -> List[Dict[str, Union[str, List[str]]]]:
        ollama_messages: List = []
        for message in messages:
            role = ""
            if isinstance(message, HumanMessage):
                role = "user"
            elif isinstance(message, AIMessage):
                role = "assistant"
            elif isinstance(message, SystemMessage):
                role = "system"
            elif isinstance(message, ToolMessage): # Added this line, needed to add this to the list of supported message types
                role = "user"
            else:
                raise ValueError("Received unsupported message type for Ollama.")

            content = ""
```


In [10]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

llm = OllamaFunctions(model="mistral", format="json")

tools=[search]
            
agent_executor = create_react_agent(llm, tools=tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="What is the weather in London?")]})

response["messages"]



TypeError: Object of type TavilySearchResults is not JSON serializable